# Calculate power analysis given Linear Model effect sizes

In [1]:
library(pwr)
suppressPackageStartupMessages(library(dplyr))

In [2]:
output_file <- file.path("results", "Plate2", "power_analysis_all_cp_features_lm_plate2.tsv")
output_dp_file <- file.path("results", "power_analysis_dp_cyto_features_lm.tsv")

## Perform power analysis for CP features

In [3]:
# Load data
lm_results_file <- file.path("results", "Plate2", "linear_model_all_cp_features.tsv")
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 1198    5


feature,r2_score,cell_count_coef,Null_coef,WT_coef
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Cytoplasm_Number_Object_Number,0.129892320,0.0236269776,0.01030461,-0.01030461
Cytoplasm_AreaShape_Area,0.227781520,-0.0130794248,-0.33243517,0.33243517
Cytoplasm_AreaShape_BoundingBoxArea,0.171188338,-0.0161349671,-0.21808162,0.21808162
Cytoplasm_AreaShape_BoundingBoxMaximum_X,0.001246299,0.0004300181,-0.03978182,0.03978182
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,0.003628518,0.0012666309,-0.07191417,0.07191417
Cytoplasm_AreaShape_BoundingBoxMinimum_X,0.004854912,0.0043512410,0.00683134,-0.00683134


In [4]:
# Load feature data (for calculating n)
data_dir <-file.path("..", "..", "..", "4_processing_features", "data", "Plate2", "CellProfiler")
cp_file <- file.path(data_dir, "nf1_sc_norm_cellprofiler_plate2.csv.gz")

cp_df <- readr::read_csv(
    cp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_WellRow="c",
        Metadata_WellCol="c",
        Metadata_Well="c",
        Metadata_gene_name="c",
        Metadata_genotype="c"
    )
)

print(dim(cp_df))
head(cp_df, 3)

New names:
• `` -> `...1`


[1] 1681 1211


...1,Metadata_WellRow,Metadata_WellCol,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,⋯,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,A,1,47,NF1,WT,5,1,A1,1,⋯,-0.4953495,-0.5140452,-0.5556103,-0.5599953,-0.5487354,-0.5557258,-0.3484580,-0.3405557,-0.3503081,-0.3460403
1,A,1,47,NF1,WT,5,1,A1,2,⋯,-0.3640100,-0.3558701,-0.5269353,-0.5195026,-0.5204616,-0.5214104,-0.3260656,-0.3214580,-0.3281426,-0.3220526
2,A,1,47,NF1,WT,5,1,A1,3,⋯,-0.5607370,-0.5620716,-0.5870021,-0.5814423,-0.5864145,-0.5869553,-0.2572193,-0.2587695,-0.2623094,-0.2488917


In [5]:
# Define constants for power analysis
n_conditions <- 2  # NF1 WT and Null
n_samples <- dim(cp_df)[1]

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- 0.05 / dim(lm_results_df)[1]
power <- 0.8

print(c(u, v))
print(sig_level)

[1]    1 1679
[1] 4.173623e-05


In [6]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (cp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!cp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [7]:
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_file)

print(dim(power_results_df))
head(power_results_df)

Warning message in (function (..., deparse.level = 1) :
“number of columns of result is not a multiple of vector length (arg 65)”
Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if
`.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”


[1] 1198    6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cytoplasm_Number_Object_Number,1,1679,4.17362270450751e-05,0.8,171.817770031987
Cytoplasm_AreaShape_Area,1,1679,4.17362270450751e-05,0.8,91.0956828315909
Cytoplasm_AreaShape_BoundingBoxArea,1,1679,4.17362270450751e-05,0.8,126.507633437838
Cytoplasm_AreaShape_BoundingBoxMaximum_X,1,1679,4.17362270450751e-05,0.8,19559.0890799048
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,1,1679,4.17362270450751e-05,0.8,6707.52129569811
Cytoplasm_AreaShape_BoundingBoxMinimum_X,1,1679,4.17362270450751e-05,0.8,5009.09868928829


## Perform power analysis for DP features

In [8]:
# Load data
lm_results_file <- file.path("results", "linear_model_dp_features.tsv")
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 3832    5


feature,r2_score,cell_count_coef,Null_coef,WT_coef
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
efficientnet_0,0.04655088,-1.313378e-02,0.098892191,-0.098892191
efficientnet_1,0.02633479,9.009381e-03,-0.246789424,0.246789424
efficientnet_2,0.32808305,-2.803389e-02,0.001363537,-0.001363537
efficientnet_3,0.02219074,5.271694e-03,-0.223836924,0.223836924
efficientnet_4,0.01098208,8.577969e-05,-0.116930972,0.116930972
efficientnet_5,0.02720802,8.035771e-03,-0.254575592,0.254575592


In [9]:
# Load feature data (for calculating n)
data_dir <-file.path("..", "..", "..", "4_processing_features", "data", "Plate1", "DeepProfiler")
dp_file <- file.path(data_dir, "nf1_sc_norm_deepprofiler_cyto.csv.gz")

dp_df <- readr::read_csv(
    dp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_Plate="c",
        Metadata_Well="c",
        Metadata_Site="c",
        Metadata_Plate_Map_Name="c",
        Metadata_DNA="c",
        Metadata_ER="c",
        Metadata_Actin="c",
        Metadata_Genotype="c",
        Metadata_Genotype_Replicate="c",
        Metadata_Model="c"
    )
)
print(dim(dp_df))
head(dp_df, 3)

[1]  256 3852


Location_Center_X,Location_Center_Y,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Plate_Map_Name,Metadata_DNA,Metadata_ER,Metadata_Actin,Metadata_Genotype,⋯,efficientnet_3830,efficientnet_3831,efficientnet_3832,efficientnet_3833,efficientnet_3834,efficientnet_3835,efficientnet_3836,efficientnet_3837,efficientnet_3838,efficientnet_3839
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
650.4225,736.7067,1,D6,3,1_D6_3,../../../../1_preprocessing_data/Corrected_Images/D6_01_1_3_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/D6_01_2_3_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/D6_01_3_3_RFP_001_IllumCorrect.tif,WT,⋯,-0.03401407,0.8595177,-0.02571327,-0.7295710,0.2672313,-0.8473657,-1.09008160,0.5667855,-0.02436295,0.2304633
949.1827,256.7347,1,F6,2,1_F6_2,../../../../1_preprocessing_data/Corrected_Images/F6_01_1_2_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_2_2_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_3_2_RFP_001_IllumCorrect.tif,WT,⋯,-0.02110868,0.1855114,-0.61222690,2.9258058,-0.9277611,-0.3043499,-0.03106468,0.6265455,-0.40805978,-1.2116158
454.4601,265.7970,1,F6,2,1_F6_2,../../../../1_preprocessing_data/Corrected_Images/F6_01_1_2_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_2_2_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_3_2_RFP_001_IllumCorrect.tif,WT,⋯,-0.15838182,-0.8102113,-0.72087693,0.1278688,1.1994534,1.9774389,-0.84440530,-0.2991374,-0.77516640,-0.2921428


In [10]:
# Define constants for power analysis (n_samples is different from CP features)
n_conditions <- 2  # NF1 WT and Null
n_samples <- dim(dp_df)[1]

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- 0.05 / dim(lm_results_df)[1]
power <- 0.8

print(c(u, v))
print(sig_level)

[1]   1 254
[1] 1.304802e-05


In [11]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (dp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!dp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[dp_feature]] <- c(dp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[dp_feature]] <- c(dp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [12]:
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_dp_file)

print(dim(power_results_df))
head(power_results_df)

[1] 3832    6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
efficientnet_0,1,254,1.30480167014614e-05,0.8,563.521954130131
efficientnet_1,1,254,1.30480167014614e-05,0.8,1009.59107700252
efficientnet_2,1,254,1.30480167014614e-05,0.8,64.8259187122834
efficientnet_3,1,254,1.30480167014614e-05,0.8,1201.40694229486
efficientnet_4,1,254,1.30480167014614e-05,0.8,2445.51410341455
efficientnet_5,1,254,1.30480167014614e-05,0.8,976.625087249966
